In [15]:
import numpy as np
import pandas as pd
import random

# Define Classes

In [16]:
class Customer:
    def __init__(self, id, lat, lon, demand):
        self.id = id
        self.lat = lat
        self.lon = lon
        self.demand = demand
        
    def distance(self, customer):
        latDis = abs(self.lat - customer.lat)
        longDis = abs(self.lon - customer.lon)
        distance = 100 * np.sqrt((latDis ** 2) + (longDis ** 2))
        return distance
    
    def __repr__(self):
        return "(" + str(self.lat) + "," + str(self.lon) + ")"

In [17]:
class Vehicle:
    def __init__(self, type):
        self.type = type
        self.customers = []
        self.rate = None
        self.capacity = None

    def addCustomer(self, customer):
        self.customers.append(customer)

    def setRateCapacity(self):
        if self.type == 'A':
            self.rate = 1.2
            self.capacity = 25
        elif self.type == 'B':
            self.rate = 1.5
            self.capacity = 30
        else:
            self.rate = 0
            self.capacity = 0
    
    def setStartPoint(self):
        self.customers.append(Customer(-1, 4.4184, 114.0932, 0))

In [18]:
class Fitness:
    def __init__(self, vehicle):
        self.vehicle = vehicle
        self.fitness = None
        self.distance = None

    
    def getDistance(self):
        route = self.vehicle.customers
        distance = 0

        for i in range (0, len(route)):
            fromCity = route[i]
            if i+1 < len(route):
                toCity = route[i+1]

            # return to depot
            else:
                toCity = route[0]
            
            distance += fromCity.distance(toCity)

        return distance

    def getCost(self):
        self.distance = self.getDistance()
        return self.distance*(self.vehicle.rate)  
    
    def getDemand(self):
        route = self.vehicle.customers
        totalDemand = 0

        for customer in route:
            totalDemand += customer.demand

        return totalDemand
            

# Generate Population

In [19]:
# Read data
data = pd.read_csv('data.csv')
data.head()

li = data.values.tolist()

customers = []
for i in li:
    customers.append(Customer(i[0], i[1], i[2], i[3]))


# Generate population
solutions = [] #array of vehicles
populationSize = 1000
for s in range(populationSize):
    print('iteration'+ str(s+1))
    customersToFulfill = customers.copy()

    vehicles = []

    while(len(customersToFulfill) !=0):
        # Randomly choose a vehicle
        type = random.choice(['A', 'B'])
        vehicle = Vehicle(type)
        vehicle.setStartPoint()
        vehicle.setRateCapacity()

        print('Chosen vehicle '+ vehicle.type)

        # Randomly fill in the routes
        filledCapacity = 0

        while(customersToFulfill):
            selectedCustomer = random.choice(customersToFulfill)
            if((filledCapacity+selectedCustomer.demand) <= vehicle.capacity):
                filledCapacity += selectedCustomer.demand
                vehicle.addCustomer(selectedCustomer)
                customersToFulfill.remove(selectedCustomer)
                print('Customer ' + str(selectedCustomer.id) + ', demand ' + str(selectedCustomer.demand))
            else:
                break
            

        vehicles.append(vehicle)

    solutions.append(vehicles)
    
solutions


iteration1
Chosen vehicle A
Customer 7.0, demand 3.0
Customer 4.0, demand 6.0
Customer 2.0, demand 8.0
Customer 8.0, demand 6.0
Chosen vehicle A
Customer 5.0, demand 5.0
Customer 3.0, demand 3.0
Customer 10.0, demand 8.0
Customer 6.0, demand 8.0
Chosen vehicle B
Customer 9.0, demand 5.0
Customer 1.0, demand 5.0
iteration2
Chosen vehicle A
Customer 6.0, demand 8.0
Customer 1.0, demand 5.0
Customer 10.0, demand 8.0
Chosen vehicle B
Customer 8.0, demand 6.0
Customer 5.0, demand 5.0
Customer 3.0, demand 3.0
Customer 4.0, demand 6.0
Customer 2.0, demand 8.0
Chosen vehicle B
Customer 7.0, demand 3.0
Customer 9.0, demand 5.0
iteration3
Chosen vehicle B
Customer 2.0, demand 8.0
Customer 8.0, demand 6.0
Customer 1.0, demand 5.0
Customer 4.0, demand 6.0
Customer 9.0, demand 5.0
Chosen vehicle A
Customer 7.0, demand 3.0
Customer 6.0, demand 8.0
Customer 10.0, demand 8.0
Customer 3.0, demand 3.0
Chosen vehicle B
Customer 5.0, demand 5.0
iteration4
Chosen vehicle B
Customer 8.0, demand 6.0
Customer

[[<__main__.Vehicle at 0x173830560c0>,
 [<__main__.Vehicle at 0x173830a5790>, <__main__.Vehicle at 0x173830a5070>],
 [<__main__.Vehicle at 0x173830a7740>, <__main__.Vehicle at 0x173830a7800>],
 [<__main__.Vehicle at 0x173830a78c0>, <__main__.Vehicle at 0x173830a7980>],
 [<__main__.Vehicle at 0x173840d4380>, <__main__.Vehicle at 0x173840d4440>],
 [<__main__.Vehicle at 0x173840d4e00>, <__main__.Vehicle at 0x173840d4ec0>],
 [<__main__.Vehicle at 0x173840d5400>, <__main__.Vehicle at 0x173840d54c0>],
 [<__main__.Vehicle at 0x173840d6300>, <__main__.Vehicle at 0x173840d63c0>],
 [<__main__.Vehicle at 0x173840d6b40>, <__main__.Vehicle at 0x173840d6c00>],
 [<__main__.Vehicle at 0x173840d75c0>, <__main__.Vehicle at 0x173840d7680>],
 [<__main__.Vehicle at 0x173840d7e00>, <__main__.Vehicle at 0x173840d7ec0>],
 [<__main__.Vehicle at 0x173840d8b00>, <__main__.Vehicle at 0x173840d8bc0>],
 [<__main__.Vehicle at 0x173840d8ec0>, <__main__.Vehicle at 0x173840d8f80>],
 [<__main__.Vehicle at 0x173840d9280>

# Selection

In [20]:
def fitness(solution):
    solutionCost = 0
    for vehicle in solution:
        fitness = Fitness(vehicle)
        cost = fitness.getCost()
        solutionCost += cost
    return solutionCost

def distance(solution):
    solutionDistance = 0
    for vehicle in solution:
        fitness = Fitness(vehicle)
        distance = fitness.getDistance()
        solutionDistance += distance
    return solutionDistance


def getParents(population):
    choices = population.copy()

    chosenSolutionCost = []
    for vehicles in choices:
        solutionCost = fitness(vehicles)
        chosenSolutionCost.append(solutionCost)


    minpos = chosenSolutionCost.index(min(chosenSolutionCost))
    minA = choices[minpos]
    chosenSolutionCost.pop(minpos)
    minpos = chosenSolutionCost.index(min(chosenSolutionCost))
    minB = choices[minpos]

    return minA, minB

A, B = getParents(solutions)
A[0].customers



[(4.4184,114.0932),
 (4.3163,114.0764),
 (4.3555,113.9777),
 (4.3184,113.9932),
 (4.3976,114.0049)]

# Crossover

In [21]:
def findMissing(li):
    return sorted(set(range(1, 11)).difference(li))

def convertToTwoDList(parent):
    output = [[],[],[]]
    outputVehicles = []
    for i, vehicle in enumerate(parent):
        if (vehicle.customers):
            # remove depot
            vehicle.customers.pop(0)
        outputVehicles.append(vehicle.type)
        # for every route
        for j in range(len(vehicle.customers)):
            output[i].append(parent[i].customers[j].id)
    return output, outputVehicles


def convertToOneDList(li):
    convertedList = []
    for i in range(len(li)):
        for j in range(len(li[i])):
            convertedList.append(int(li[i][j]))
    return convertedList



In [22]:

def getCrossoverOffspring(firstParent, secondParent):
    childVehicles = []

    first, firstVehicles = convertToTwoDList(firstParent)
    second, secondVehicles = convertToTwoDList(secondParent)


    # Random Choice of 1,2,3,4
    choice = random.randint(1, 4)
    # Case 1 - first parent is  main, two vehicles remain, one vehicle changes
    if (choice == 1 or choice == 2):
        if (choice == 1):
            parent = first.copy()
            secondary = second.copy()
            selectedVehicleIdx = random.randint(0, len(parent)-1)
            selectedToSwapVehicle = random.choice(secondary)
            parent[selectedVehicleIdx] = selectedToSwapVehicle
            parent = convertToOneDList(parent)
            childVehicles = firstVehicles.copy()

        # Case 2 - second parent is main, two vehicles remain, one vehicle changes
        elif (choice == 2):
            parent = second.copy()
            secondary = first.copy()
            selectedVehicleIdx = random.randint(0, len(parent)-1)
            selectedToSwapVehicle = random.choice(secondary)
            parent[selectedVehicleIdx] = selectedToSwapVehicle
            parent = convertToOneDList(parent)
            childVehicles = secondVehicles.copy()

        missing = findMissing(parent)

        child = []
        for i in parent:
            if i not in child:
                child.append(i)
            else:
                if(missing):
                    child.append(missing[0])
                    missing.pop(0)

        if (missing):
            child += missing


    elif (choice == 3 or choice == 4):
        # Case 3 - take first parent as a whole, no crossover, direcly sent to mutation
        if (choice == 3):
            child = convertToOneDList(first)
            childVehicles = firstVehicles.copy()
        # Case 4 - take second parent as a whole, no crossover, direcly sent to mutation
        elif (choice == 4):
            child = convertToOneDList(second)
            childVehicles = secondVehicles.copy()

    return child, childVehicles

print(A[0].customers)
child = getCrossoverOffspring(A, B)
child


[(4.4184,114.0932), (4.3163,114.0764), (4.3555,113.9777), (4.3184,113.9932), (4.3976,114.0049)]


([3, 1, 4, 2, 5, 10, 9, 6, 7, 8], ['A', 'A', 'A'])

# Mutation

In [23]:
# Vehicle Mutation - Bit Flip - Rate: 0.5
def vehicleMutation(li):
    prob = random.randint(0, 100)
    if prob <= 50:
        idx = random.randint(0, len(li)-1)
        if li[idx] == 'A':
            li[idx] = 'B'
        else:
            li[idx] = 'A'
    
    return li

# Routes Mutation - Swap - Rate: 0.7
def routeMutation(li):
    prob = random.randint(0, 100)
    if prob <= 70:
        if(len(li) >= 1):
            aIdx = random.randint(0, len(li)-1)
            bIdx = random.randint(0, len(li)-1)

            temp = li[aIdx]
            li[aIdx] = li[bIdx]
            li[bIdx] = temp

    return li

In [24]:
def getMutatedOffspring(first, second):

    child, childVehicles = getCrossoverOffspring(first, second)
    mutatedChild = routeMutation(child)
    mutatedVehicles = vehicleMutation(childVehicles)

    return mutatedChild, mutatedVehicles




# Convert back to class

In [25]:
# convert routes to Customer class
data = pd.read_csv('data.csv')
data.head()

li = data.values.tolist()
print(li)

customers = []
for i in li:
    customers.append(Customer(i[0], i[1], i[2], i[3]))


[[1.0, 4.3555, 113.9777, 5.0], [2.0, 4.3976, 114.0049, 8.0], [3.0, 4.3163, 114.0764, 3.0], [4.0, 4.3184, 113.9932, 6.0], [5.0, 4.4024, 113.9896, 5.0], [6.0, 4.4142, 114.0127, 8.0], [7.0, 4.4804, 114.0734, 3.0], [8.0, 4.3818, 114.2034, 6.0], [9.0, 4.4935, 114.1828, 5.0], [10.0, 4.4932, 114.1322, 8.0]]


In [26]:
def getOffspring(first, second):
    mutatedChild, mutatedVehicles = getMutatedOffspring(first, second)
    while(len(mutatedChild) != 10):
        mutatedChild, mutatedVehicles = getMutatedOffspring(first, second)
    # print(mutatedChild)
    # print(mutatedVehicles)

    offspring = []
    for i in mutatedVehicles:
        vehicle = Vehicle(i)
        vehicle.setStartPoint()
        vehicle.setRateCapacity()

        filledCapacity = 0

        while(mutatedChild):
            idx = mutatedChild[0]
            selectedCustomer = customers[idx-1]
            if((filledCapacity+selectedCustomer.demand) <= vehicle.capacity):
                filledCapacity += selectedCustomer.demand
                vehicle.addCustomer(selectedCustomer)
                mutatedChild.pop(0)
            else:
                break
        
        offspring.append(vehicle)

    return offspring
    

# Main function

In [27]:
import copy

parent1, parent2 = getParents(solutions)


noIteration = 500
minScore = 1000

for i in range(noIteration):
    population = []
    # each parent will produce 1000 offspring, and two best are selected
    for j in range(500):
        offspringA = getOffspring(parent1, parent2)
        offspringB = getOffspring(parent1, parent2)

        population.append(offspringA)
        population.append(offspringB)
    

    parent1, parent2 = getParents(population)

    averageScore = fitness(parent1)

    print('Iteration: ' + str(i+1))
    print('Score: ' + str(averageScore))

    if(averageScore < minScore):
        minScore = averageScore
        minParent = copy.deepcopy(parent1)



Iteration: 1
Score: 143.42708299858427
Iteration: 2
Score: 143.42708299858427
Iteration: 3
Score: 143.42708299858427
Iteration: 4
Score: 143.42708299858427
Iteration: 5
Score: 143.42708299858427
Iteration: 6
Score: 143.42708299858427
Iteration: 7
Score: 143.42708299858427
Iteration: 8
Score: 143.42708299858427
Iteration: 9
Score: 143.42708299858427
Iteration: 10
Score: 143.42708299858427
Iteration: 11
Score: 143.42708299858427
Iteration: 12
Score: 143.42708299858427
Iteration: 13
Score: 143.42708299858427
Iteration: 14
Score: 143.42708299858427
Iteration: 15
Score: 143.42708299858427
Iteration: 16
Score: 143.42708299858427
Iteration: 17
Score: 143.42708299858427
Iteration: 18
Score: 143.42708299858427
Iteration: 19
Score: 143.42708299858427
Iteration: 20
Score: 143.42708299858427
Iteration: 21
Score: 143.42708299858427
Iteration: 22
Score: 143.42708299858427
Iteration: 23
Score: 143.42708299858427
Iteration: 24
Score: 143.42708299858427
Iteration: 25
Score: 143.42708299858427
Iteration

In [28]:
print("Total Distance: " + str(round(distance(minParent), 2)) + " km")
print("Total Cost: RM " + str(round(minScore, 2)))
print('')


for idx, vehicle in enumerate(minParent):
    if(vehicle):

        print(f"Vehicle {idx+1} (Type {vehicle.type})")
        vehicleFitness = Fitness(vehicle)
        print(f"Round Trip Distance: {round(vehicleFitness.getDistance(),3)} km, Cost: RM {round(vehicleFitness.getCost(),2)}, Demand: {vehicleFitness.getDemand()}")

        for idx, route in enumerate(vehicle.customers):
            
            if (route.id == -1):
                routeName = "Depot"
                print(f"{routeName} -> ")
            else:
                routeName = "C"+str(int(route.id))
                print(f"{routeName} ({round(route.distance(vehicle.customers[idx-1]),2)} km) -> ")

        backToDepotDistance = vehicle.customers[-1].distance(vehicle.customers[0])
        print(f"Depot ({round(backToDepotDistance, 2)} km)")
        print('')

Total Distance: 111.95 km
Total Cost: RM 134.34

Vehicle 1 (Type A)
Round Trip Distance: 49.986 km, Cost: RM 59.98, Demand: 22.0
Depot -> 
C5 (10.48 km) -> 
C6 (2.59 km) -> 
C7 (8.98 km) -> 
C8 (16.32 km) -> 
Depot (11.61 km)

Vehicle 2 (Type A)
Round Trip Distance: 36.775 km, Cost: RM 44.13, Demand: 22.0
Depot -> 
C2 (9.07 km) -> 
C1 (5.01 km) -> 
C4 (4.02 km) -> 
C3 (8.32 km) -> 
Depot (10.35 km)

Vehicle 3 (Type A)
Round Trip Distance: 25.187 km, Cost: RM 30.22, Demand: 13.0
Depot -> 
C9 (11.69 km) -> 
C10 (5.06 km) -> 
Depot (8.44 km)

